<a href="https://colab.research.google.com/github/AstridSerruto/Projects/blob/master/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset from https://files.grouplens.org/datasets/movielens/ml-25m.zip

We are creating a mobie recommendation system using Python and pandas. The goal is to type the name of a movie into an input box and obtain a recommedation for other movies we might like.

In [2]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
import re

from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
from google.colab import drive
drive.mount('/content/drive')
	
movies = pd.read_csv('/content/drive/My Drive/Datasets/movielens/movies.csv')

Mounted at /content/drive


In [4]:
ratings = pd.read_csv('/content/drive/My Drive/Datasets/movielens/ratings.csv')

In [5]:
#Clean title to make search easier
def clean_title(title):
  return re.sub("[^a-zA-Z0-9 ]", "", title) #Remove characters that fit a certain pattern

In [6]:
movies['clean_title'] = movies['title'].apply(clean_title)

movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [7]:
# Initialize vectorizer with ngram
vectorizer = TfidfVectorizer(ngram_range=(1,2))
# Convert titles into matrix
tfidf = vectorizer.fit_transform(movies['clean_title'])

In [8]:
#Compute the similarity of terms entered and movies
def search(title):
  title = clean_title(title)
  query_vec = vectorizer.transform([title])
  similarity = cosine_similarity(query_vec, tfidf).flatten()
#Find titles with greates similarity with search terms
  indices = np.argpartition(similarity, -5)[-5:]
  results = movies.iloc[indices][::-1]
  return results

In [9]:
#Create input box
movie_input = widgets.Text(
  value='Toy Story',
  description='Movie Title:',
  disabled=False
)
movie_list = widgets.Output()

#Create Output widget, search for title and display
def on_type(data):
  with movie_list:
    movie_list.clear_output()
    title = data['new']
    if len (title) > 5:
      display(search(title))
      
#Connect input to output
movie_input.observe(on_type, names='value')

display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [10]:
#Find movies similar to the ones we like
ratings 

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [11]:
movie_id = 1

In [12]:
#Finding users who like the same movies
similar_users = ratings[(ratings['movieId'] == movie_id) & (ratings['rating'] >= 5)]['userId'].unique()

similar_users

array([    36,     75,     86, ..., 162518, 162519, 162530])

In [13]:
#Find the movies other users like
similar_user_recs = ratings[(ratings['userId'].isin(similar_users)) & (ratings['rating'] > 4)]['movieId']

similar_user_recs

5101           1
5105          34
5111         110
5114         150
5127         260
            ... 
24998388    3706
24998389    3735
24998391    3763
24998392    4187
24998393    4321
Name: movieId, Length: 912084, dtype: int64

In [14]:
#Find only the movies that >10% of users liked
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .1]

In [15]:
similar_user_recs

1       1.000000
318     0.414556
260     0.404561
356     0.347253
296     0.342663
          ...   
1259    0.102991
7361    0.101881
1206    0.101362
1307    0.101066
1208    0.100918
Name: movieId, Length: 92, dtype: float64

In [16]:
#How much do all users like movies
all_users = ratings[(ratings['movieId'].isin(similar_user_recs.index)) & (ratings['rating'] > 4)]

all_users


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
29,1,4973,4.5,1147869080
48,1,7361,5.0,1147880055
72,2,110,5.0,1141416589
76,2,260,5.0,1141417172
...,...,...,...,...
25000055,162541,4973,4.5,1240950790
25000057,162541,4993,5.0,1240952610
25000065,162541,5952,5.0,1240952617
25000078,162541,7153,5.0,1240952613


In [17]:
#What percent of all users recomend these movies
all_users_recs = all_users['movieId'].value_counts() / len(all_users['userId'].unique())

In [18]:
all_users_recs

318      0.345282
296      0.287220
2571     0.246217
356      0.237370
593      0.227930
           ...   
1387     0.047886
1307     0.046195
745      0.037362
78499    0.035445
2355     0.025316
Name: movieId, Length: 92, dtype: float64

In [19]:
#Compare percentages
rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
rec_percentages.columns = ['similar', 'all']

rec_percentages

,similar,all
1,1.000000,0.125844
32,0.157856,0.101190
34,0.158226,0.052696
47,0.203613,0.145762
50,0.262180,0.202306
...,...,...
58559,0.160743,0.147779
60069,0.137272,0.076990
68954,0.136310,0.065525
78499,0.138161,0.035445


In [20]:
#Create score
rec_percentages['score']= rec_percentages['similar'] / rec_percentages['all']

In [21]:
rec_percentages = rec_percentages.sort_values('score', ascending=False)

In [22]:
rec_percentages

,similar,all,score
1,1.000000,0.125844,7.946323
3114,0.295498,0.054186,5.453383
2355,0.124685,0.025316,4.925186
78499,0.138161,0.035445,3.897906
588,0.233674,0.068117,3.430480
...,...,...,...
58559,0.160743,0.147779,1.087725
79132,0.129424,0.132559,0.976349
7361,0.101881,0.105172,0.968704
2959,0.205020,0.218656,0.937638


In [24]:
#Take top ten recomendations and merge with movie data
rec_percentages.head(10).merge(movies, left_index=True, right_on='movieId')


,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.125844,7.946323,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,0.295498,0.054186,5.453383,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
2264,0.124685,0.025316,4.925186,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Bugs Life A 1998
14813,0.138161,0.035445,3.897906,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
580,0.233674,0.068117,3.430480,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin 1992
587,0.198949,0.060514,3.287671,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast 1991
33,0.158226,0.052696,3.002602,34,Babe (1995),Children|Drama,Babe 1995
4780,0.210647,0.071444,2.948410,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,Monsters Inc 2001
1047,0.143418,0.049202,2.914882,1073,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical,Willy Wonka the Chocolate Factory 1971
729,0.108322,0.037362,2.899227,745,Wallace & Gromit: A Close Shave (1995),Animation|Children|Comedy,Wallace Gromit A Close Shave 1995


In [25]:
def find_similar_movies(movie_id):
  similar_users = ratings[(ratings['movieId'] == movie_id) & (ratings['rating'] >= 5)]['userId'].unique()
  similar_user_recs = ratings[(ratings['userId'].isin(similar_users)) & (ratings['rating'] > 4)]['movieId']

  similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
  similar_user_recs = similar_user_recs[similar_user_recs > .1]

  all_users = ratings[(ratings['movieId'].isin(similar_user_recs.index)) & (ratings['rating'] > 4)]
  all_users_recs = all_users['movieId'].value_counts() / len(all_users['userId'].unique())

  rec_percentages['score']= rec_percentages['similar'] / rec_percentages['all']

  rec_percentages = rec_percentages.sort_values('score', ascending=False)
  return rec_percentages.head(10).merge(movies, left_index=True, right_on='movieId')[['score', 'title', 'genres']]

In [28]:
#Build widget
movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disable=False
)

recommendation_list = widgets.Output()

def on_type(data):
  with recommendation_list:
    recommendation_list.clear_output()
    title = data['new']
    if len(title) > 5:
      results = search(title)
      movie_id = results.iloc[0]['movieId']
      display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()